## TSP

Here we solve it as as MIP
[traveling salesman problem TSP](https://en.wikipedia.org/wiki/Travelling_salesman_problem)

In [1]:
from pulp import *
import numpy as np

In [2]:
sites = ['org','A','B','C','D','E','F','G']

In [3]:
#non symetric distances
distances ={}
for a in sites:
    for b in sites:
        if a==b:
            continue
        
        distances[(a,b)] = np.random.randint(10,50)
        

In [4]:
#we need to keep track of the order in the tour to eliminate the possibility of subtours
u = LpVariable.dicts('u', sites, 0, len(sites)-1, LpInteger)

In [5]:
#we need lots of binary variables to indicate if a city i is connected to city j
x = LpVariable.dicts('x',distances, 0,1,LpBinary)

In [6]:
prob=LpProblem("salesman",LpMinimize)

In [7]:
cost = lpSum([x[(i,j)]*distances[(i,j)] for (i,j) in distances])
prob+=cost

In [8]:
#constraints
for k in sites:
    #every city has exactly one inbound connection
    prob+= lpSum([ x[(i,k)] for i in sites if (i,k) in x]) ==1
    #every city has exactly one outbound connection
    prob+=lpSum([ x[(k,i)] for i in sites if (k,i) in x]) ==1
    
#subtour elimination
M=len(sites)+1
for i in sites:
    for j in sites:
        if i != j and (i != 'org' and j!= 'org') and (i,j) in x:
            prob += u[i] - u[j] + M*x[(i,j)] <= M - 1

In [9]:
%time prob.solve()

CPU times: user 5.1 ms, sys: 3.81 ms, total: 8.91 ms
Wall time: 63.9 ms


1

In [10]:
for (i,j) in distances:
    v = x[(i,j)].varValue
    if v >0:
        print(i,j, v, distances[(i,j)] )

B org 1.0 12
org D 1.0 14
D F 1.0 13
E A 1.0 20
C G 1.0 16
A B 1.0 21
G E 1.0 13
F C 1.0 12


In [11]:
sites_left = sites.copy()
org = 'org'
tour=[]
tour.append(sites_left.pop( sites_left.index(org)))

while len(sites_left) > 0:
    
    for k in sites_left:
        if x[(org,k)].varValue ==1:
            tour.append( sites_left.pop( sites_left.index(k)))
            org=k
            break
            
tour.append('org')

tour_legs = [distances[(tour[i-1], tour[i])] for i in range(1,len(tour))]

print(' -> '.join(tour))

org -> D -> F -> C -> G -> E -> A -> B -> org


In [12]:
sum(tour_legs)

121